In [1]:
from bokeh.io import output_notebook, show, curdoc
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, BasicTickFormatter, HoverTool, Legend, Column, Div
from bokeh.models.widgets import Tabs, Panel
from bokeh.palettes import Category20
from bokeh.layouts import column, row, gridplot
from bokeh.transform import cumsum
import pandas as pd
pd.options.mode.chained_assignment = None 
from math import pi

output_notebook()

csv_url = "https://health-infobase.canada.ca/src/data/covidLive/covid19.csv"

df = pd.read_csv(csv_url)

#parse government data so it is usable
df['dates'] = pd.to_datetime(df['date'], dayfirst = True)
df['YMD'] = df['dates'].dt.date
df = df.drop(['date', 'dates', 'prnameFR', 'pruid'], axis = 1)
#if provinces don't report active cases, assume they have 0, so the data will plot
df['numactive'].fillna(0.0, inplace = True)


#List of unique provinces, Canada, ordered alphabetically with Canada first
provnames = df.prname.unique()
provnames = list(provnames)
provnames.pop()
provnames.sort()
myorder = [2, 0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
provnames = [provnames[i] for i in myorder]

#define colour palette
colors = (list(Category20[14]))

#function to make province-wise dataset
def make_dataset(pnames):
    
    by_prov = pd.DataFrame()

    # Iterate through all the provinces
    for i in pnames:

        # Subset to the province
        sub = df[df['prname'] == i]

        # Add to the overall dataframe
        by_prov = by_prov.append(sub)
    
    return by_prov

#to make barchart dataset, must retrieve data from most recent date only
def make_barchart_dataset(pnames):
    df_sub = df[df['prname'].isin(provnames)]
    recent = max(df_sub['YMD'])
    recent_date = df_sub[df_sub['YMD'] == recent]

    return recent_date

#must pivot datasets for linear plot data so prov names are columns
def format_dataset(df, value):
    df_pivot = pd.pivot_table(df,
    columns = 'prname',
    index = 'YMD',
    values = value)
    
    df_pivot = df_pivot.rename_axis(None)

    return df_pivot

#function to make linear plots
def plot_lines_multi(src, title, yaxisname, tooltips, yaxistype = "linear"):

    numlines = len(src)

    #import color pallete
    mypalette = colors[0:numlines]

    # make a list of our columns
    col = []
    [col.append(i) for i in src.columns]
    
    #source = ColumnDataSource(df)

    # make the figure, 
    p = figure(x_axis_type="datetime", title= title, width = 600, height = 600, 
              tools='pan, wheel_zoom, reset', y_axis_type = yaxistype)
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = yaxisname

    # loop through our columns and colours
    for (columnnames, colore) in zip(col, mypalette):
        p.line(src.index, src[columnnames], legend_label = columnnames, color = colore, line_width=4, alpha = 0.8)

    #format axes and legend
    p.xaxis.major_label_orientation = "vertical"
    p.yaxis.formatter = BasicTickFormatter(use_scientific=False)
    p.legend.location = "top_left"
    
    # Hover tool
    hover = HoverTool(mode = 'mouse')
    hover.tooltips = tooltips
    hover.formatters = { "$x": "datetime"}
        
    p.add_tools(hover)
    p.legend.click_policy = 'hide'

    output_file("Canada_COVID-19_Dashboard.html", title="Canada COVID-19 Dashboard")

    return p 
    

#function to make bar plots
def plot_bar_multi(src, yaxis, title, yaxisname, cols):
    numlines = len(src)

    #import color pallete
    mypalette = colors[0:numlines]

    # make a list of our columns
    col = []
    [col.append(i) for i in src.columns]
    
    source = ColumnDataSource(data = src)
    regions = source.data['prname'].tolist()

    # make the figure, 
    b = figure(title= title, width = 600, height = 600, 
              tools='pan, wheel_zoom, reset', x_range = regions)
    b.xaxis.axis_label = 'Region'
    b.yaxis.axis_label = yaxisname
    
    b.vbar(x = 'prname', top = yaxis, source = source, alpha = 0.8, width = 0.8)

    b.xaxis.major_label_orientation = "vertical"

    # Hover tool
    hover = HoverTool()

    hover.tooltips = [(c, '@' + c) for c in cols]

    b.add_tools(hover)

    return b 

#function to make pie plots
def plot_pie(src):
    
    cols = (list(Category20))
    # remove canada
    src_provs = src[src['prname'] != 'Canada']
    #src_provs['numactive'].fillna(0.0, inplace = True)
    total_active = src_provs['numactive'].sum()
    src_provs['frac_active'] = src_provs['numactive'] / total_active
    src_provs['angle'] = src_provs['frac_active'] / src_provs['frac_active'].sum() * 2 * pi
    src_provs['color'] = Category20[len(src_provs)]
    
    pie = figure (plot_height = 600, plot_width = 600, title = 'Currently Active Cases By Region', 
                tools = "hover, pan, wheel_zoom, reset", tooltips = "@prname: @numactive")
    
    pie.wedge(x=0, y=1, radius=0.5,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='prname', source=src_provs)
    
    pie.legend.location = "top_left"
    pie.axis.visible = False
    
    return(pie)


#Make Plots
#Total Cases
front = Div(text="""
<h1>Canada COVID-19 Dashboard</h1>\n
<h3>This interactive dashboard is updated daily with data from <a href="https://health-infobase.canada.ca/">Canada's Public Health Infobase</a>.</h3><br><br>
<h4>It was built using <a href= "https://www.python.org/">python</a>, <a href="https://pandas.pydata.org/">pandas</a> and <a href="https://bokeh.org/">bokeh</a>.</h4><br><br>
You can interact with the dashboard in the following ways:
<ul>
  <li><strong>Hover</strong> over a line, bar or segment of a pie chart with the mouse to view detailed information.</li> 
  <li><strong>Click</strong> on plot legends to hide lines.</li>
  <li><strong>Zoom in or pan</strong> to manipulate the area of a plot that you are viewing.</li>
</ul>
""",
width=600, height=600)
src = format_dataset(make_dataset(provnames), 'numtotal')
p = plot_lines_multi(src, 'Total Cases', 'Cases', [('Date', '$x{%F}'), ('Cases', '$y{use_scientific = False}')],
                    "log")
layout_front = row(p, front)

tab_front = Panel(child=layout_front, title = 'Total Cases')

#New Cases
src_bar = make_barchart_dataset(provnames)
b1 = plot_bar_multi(src_bar, 'numtoday', 'Number of Cases Identified Today', 'Cases', ['prname', 'numtoday'])
b4 = plot_bar_multi(src_bar, 'percentoday', 'Percent of Cases Identified Today', 'Percent (%)', ['prname', 'percentoday'])
layout = row(b1, b4)

tab = Panel(child=layout, title = 'New Cases')

#Active Cases By Region
#layout = [percent_active, num_active]
pie = plot_pie(src_bar)
src6 = format_dataset(make_dataset(provnames), 'numactive')
print(src6.head())
p6 = plot_lines_multi(src6, 'Number of Cases Active Historically', 'Cases', 
                      [('Date', '$x{%F}'), ('Cases', '$y{use_scientific = False}')], "log")
layout_cases = row(pie, p6)
tab_cases = Panel(child = layout_cases, title = 'Active Cases by Region')


#Recovered Cases
src1 = format_dataset(make_dataset(provnames), 'numrecoveredtoday')
p1 = plot_lines_multi(src1, 'Total Recoveries', 'Recoveries', [('Date', '$x{%F}'), ('Recoveries', '$y{use_scientific = False}')])
b2 = plot_bar_multi(src_bar, 'numrecoveredtoday', 'Recoveries Today', 'Recoveries', ['prname', 'numrecoveredtoday'])
layout1 = row(p1, b2)
tab1 = Panel(child=layout1, title = 'Recoveries')

#Deaths and Mortality
src2 = format_dataset(make_dataset(provnames), 'numdeathstoday')
p2 = plot_lines_multi(src2, 'Number of Deaths by Date', 'Deaths', [('Date', '$x{%F}'), 
                                                                   ('Deaths', '$y{use_scientific = False}')])
b3 = plot_bar_multi(src_bar, 'numdeathstoday', 'Deaths Today', 'Deaths', ['prname', 'numdeathstoday'])
src4 = format_dataset(make_dataset(provnames), 'percentdeath')
p4 = plot_lines_multi(src4, 'Historical Percent Mortality', 'Percent (%)', 
                      [('Date', '$x{%F}'), ('Percent Mortality', '$y{use_scientific = False}')])
srcdeaths = format_dataset(make_dataset(provnames), 'numdeaths')
pdeaths = plot_lines_multi(srcdeaths, 'Cumulative Number of Deaths', 'Deaths', [('Date', '$x{%F}'), 
                                                                   ('Deaths', '$y{use_scientific = False}')], "log")
layout2 = gridplot([[pdeaths, b3], [p2, p4]])
tab2 = Panel(child=layout2, title = 'Deaths')

#Case rate per 100,000 population
src3 = format_dataset(make_dataset(provnames), 'ratetotal')
p3 = plot_lines_multi(src3, 'Rate of Infection', 'Rate of Infection (per 100,000 population)',
                     [('Date', '$x{%F}'), ('Rate of Infection', '$y{use_scientific = False}')])
layout3 = row(p3)
tab3 = Panel(child=layout3, title = 'Rate of Infection')

#Testing Rate per 1 million population
src5 = format_dataset(make_dataset(provnames), 'ratetested')
p5 = plot_lines_multi(src5, 'Testing Rate', 'Testing Rate (per 1 million population)',
                     [('Date', '$x{%F}'), ('Testing Rate', '$y{use_scientific = False}')])
layout4 = row(p5)
tab4 = Panel(child=layout4, title = 'Testing Rate')

#Assemble layout for all tabs
tabs = Tabs(tabs=[tab_front, tab, tab_cases, tab1, tab2, tab3, tab4])

# Add it to the current document (displays plot)
curdoc().add_root(tabs)
show(tabs)

Loading BokehJS ...

prname      Alberta  British Columbia  Canada  Manitoba  New Brunswick  \
2020-01-31      NaN               1.0     4.0       NaN            NaN   
2020-02-08      NaN               4.0     7.0       NaN            NaN   
2020-02-16      NaN               5.0     8.0       NaN            NaN   
2020-02-21      NaN               6.0     9.0       NaN            NaN   
2020-02-24      NaN               6.0    10.0       NaN            NaN   

prname      Newfoundland and Labrador  Northwest Territories  Nova Scotia  \
2020-01-31                        NaN                    NaN          NaN   
2020-02-08                        NaN                    NaN          NaN   
2020-02-16                        NaN                    NaN          NaN   
2020-02-21                        NaN                    NaN          NaN   
2020-02-24                        NaN                    NaN          NaN   

prname      Nunavut  Ontario  Prince Edward Island  Quebec  Saskatchewan  \
2020-01-31      